In [ ]:
import numpy as np
import pprint
import sys
import gym
try:
    import Box2D
except ModuleNotFoundError:
    print("Box2D not found !!")
    !pip install Box2D-py
    import Box2D
try:
    import gym_gridworlds
except ModuleNotFoundError:
    print("gym_gridworlds not found. so Installing rn ")
    !pip install gym_gridworlds
    import gym_gridworlds
class DiscreteActions(gym.ActionWrapper):
    def __init__(self, env, disc_to_cont):
        super().__init__(env)
        self.disc_to_cont = disc_to_cont

        from gym.spaces import Discrete
        self.action_space = Discrete(len(disc_to_cont))

    def action(self, act):
        return self.disc_to_cont[act]
if __name__ == "__main__":

    import numpy as np
    env = gym.make("LunarLanderContinuous-v2")
    wrapped_env = DiscreteActions(env, [np.array([1,0]), np.array([-1,0]),
                                        np.array([0,1]), np.array([0,-1])])
    print(wrapped_env.action_space)

In [ ]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

In [ ]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [ ]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    def one_step_lookahead(state, V):
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A
    V = np.zeros(env.nS)
    while True:
        delta = 0
        for s in range(env.nS):
            A = one_step_lookahead(s, V)
            best_action_value = np.max(A)
            delta = max(delta, np.abs(best_action_value - V[s]))
            V[s] = best_action_value
        if delta < theta:
            break
    # this is used to create a deterministic policy ie invloves less probablity using the optimal value func
    policy = np.zeros([env.nS, env.nA])
    for s in range(env.nS):
        A = one_step_lookahead(s, V)
        best_action = np.argmax(A)
        policy[s, best_action] = 1.0

    return policy, V

In [ ]:
policy, v = value_iteration(env)

print("Policy probability distribution:")
print(policy)
print("")

print("Reshaped grid olicy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value func:")
print(v)
print("")

print("Reshaped grid value func:")
print(v.reshape(env.shape))
print("")

In [ ]:
# Testing few  the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)